In [ ]:
import requests
import csv
import pandas as pd
from google.colab import drive
import ast
import re

# Подключаем Google Диск
drive.mount('/content/drive')
'''
# Замените на ваш ключ проекта
project_key = 'mariia10_sast-suit2'
page_size = 500

# Список для хранения отформатированных данных
formatted_issues = []

# Запрашиваем общее количество проблем
total_issues_url = f'https://sonarcloud.io/api/issues/search?componentKeys={project_key}'
total_issues_response = requests.get(total_issues_url)
total_issues = total_issues_response.json().get('total', 0)  # Получаем общее количество проблем
total_pages = (total_issues + page_size - 1) // page_size  # Округление вверх

rules_mapping = {}


for page_index in range(1, total_pages + 1):
    rules_url = f'https://sonarcloud.io/api/rules/search?languages=java&organization=mariia10&ps={page_size}&p={page_index}'
    rules_response = requests.get(rules_url)

    if rules_response.status_code == 200:
        rules_data = rules_response.json()

        for rule in rules_data.get('rules', []):
            rule_key = rule['key']
            security_standards = rule.get('securityStandards', [])

            rules_mapping[rule_key.replace("java:", "").replace("javasecurity:", "")] = security_standards  # Сохраняем ключ правила и его securityStandards
    else:
        print(f"Ошибка при запросе правил: {rules_response.status_code}")

print(rules_mapping)

# Запрашиваем данные для каждой страницы
for page_index in range(1, total_pages + 1):
    url = f'https://sonarcloud.io/api/issues/search?componentKeys={project_key}&ps={page_size}&p={page_index}&types=VULNERABILITY&languages=java'

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        print(data)
        if 'issues' in data:
            for issue in data['issues']:
                component = issue['component']
                if (component.startswith('mariia10_sast-suit2') and
                 issue['type'] == 'VULNERABILITY'):
                    test_name = component.split('/')[-1]  # Имя файла
                    category = issue['rule'].replace("java:", "").replace("javasecurity:", "")
                    line = issue['line']  #
                    real_vulnerability = 'true'
                    cwe = issue['tags'][0] if issue['tags'] else 'N/A'  # CWE (если есть)

                    # Получаем security standards для текущего правила
                    security_standards = rules_mapping.get(category, [])

                    formatted_issue = {
                        'Test Name': test_name,
                        'Category': category,
                        'Line': line,
                        'Real Vulnerability': real_vulnerability,
                        'Severity': issue['severity'],
                        'Impacts' : issue['impacts'],
                        'Security Standards': ', '.join(security_standards)  # Преобразуем список в строку
                    }
                    formatted_issues.append(formatted_issue)
    else:
        print(f"Ошибка при запросе данных: {response.status_code}")

# Сохранение в CSV файл на Google Диск
output_path = '/content/drive/My Drive/security_shepherd_sonar.csv'  # Укажите путь к вашему Google Диску
with open(output_path, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['Test Name', 'Category', 'Line', 'Real Vulnerability', 'Severity', 'Impacts', 'Security Standards'])
    writer.writeheader()  # Заголовки
    writer.writerows(formatted_issues)  # Запись данных

print(f"Отчет сохранен в {output_path} с {len(formatted_issues)} проблемами.")

'''
# Загрузка данных из CSV файлов
formatted_issues_path = '/content/drive/My Drive/security_shepherd_sonar.csv'
real_vulnerabilities_path = '/content/drive/My Drive/output.csv'  # Укажите путь к вашему файлу с реальными уязвимостями

# Попробуйте разные параметры
# Чтение реальных уязвимостей
with open(real_vulnerabilities_path, mode='r', encoding='utf-8') as real_vul_file:
    real_vulnerabilities = list(csv.reader(real_vul_file, delimiter=';'))
    # Убираем заголовки
    real_vulnerabilities = real_vulnerabilities[1:]  # Пропускаем первую строку

# Чтение форматированных проблем
with open(formatted_issues_path, mode='r', encoding='utf-8') as formatted_issues_file:
    formatted_iss = list(csv.reader(formatted_issues_file, delimiter=';'))
    # Убираем заголовки
    formatted_iss = formatted_iss[1:]  # Пропускаем первую строку

#formatted_issues = pd.DataFrame(formatted_iss)
formatted_issues = formatted_iss

# Вывод названий колонок и размера
print("Размер real_vulnerabilities:", len(real_vulnerabilities))

# Фильтрация реальных уязвимостей

# Инициализация счетчиков
TP = 0  # True Positives
FP = 0  # False Positives
FN = 0  # False Negatives

result = {}
negative = {}
for real_issue in real_vulnerabilities:
  cleaned_string = str(real_issue).strip("[]'")
  data_list = cleaned_string.split(',')
  path = data_list[1];
  pattern = r"([^/]+\.java)$"

#  Используем метод search
  matcher = re.search(pattern, path)
  benchmark_name = ""
  if matcher:
    benchmark_name = matcher.group(1)
  result[benchmark_name] = []
  negative[benchmark_name] = []

# Сравнение данных
for real_issue in real_vulnerabilities:
    # Удаление квадратных скобок и лишних пробелов
    cleaned_string = str(real_issue).strip("[]'")
    # Разделение строки на элементы списка
    data_list = cleaned_string.split(',')
    path = data_list[1];
    pattern = r"([^/]+\.java)$"

#  Используем метод search
    matcher = re.search(pattern, path)
    benchmark_name = ""
    if matcher:
      benchmark_name = matcher.group(1)
    #print(benchmark_name)
    rule = data_list[0]
    line = data_list[2]
    # Проверяем, есть ли соответствующая запись в formatted_issues
    matching_issues = []
    not_mat_issues = []
    for formatted_issue in formatted_issues:
      #print(formatted_issue)
      cleaned_string = str(formatted_issue)[1:-1]
      #cleaned_string = str(formatted_issue).strip("[]'")
      # Разделение строки на элементы списка
      form_list = cleaned_string.split(',')
      if form_list[0].replace("'","")==benchmark_name and rule in str(formatted_issue):
      #and int(line)==int(form_list[2]) :
        TP += 1

# Дополнительный блок для нахождения ложных отрицаний
for real_issue in real_vulnerabilities:
    cleaned_string = str(real_issue).strip("[]'")
    data_list = cleaned_string.split(',')
    path = data_list[1]
    pattern = r"([^/]+\.java)$"

    matcher = re.search(pattern, path)
    benchmark_name = ""
    if matcher:
        benchmark_name = matcher.group(1)

    rule = data_list[0]
    line = data_list[2]

    # Проверяем, есть ли это реальное нарушение среди отформатированных
    #and int(line) == int(str(formatted_issue).split(',')[2])
    if not any(
        (rule in str(formatted_issue) and str(formatted_issue).split(',')[0].replace("'", "") == benchmark_name)
        for formatted_issue in formatted_issues
    ):
        # Если не найдено в formatted_issues, добавляем к ложным отрицаниям
        FN += 1

for formatted_issue in formatted_issues:
    cleaned_string = str(formatted_issue).strip("[]'")
    form_list = cleaned_string.split(',')

    benchmark_name = form_list[0].replace("'", "")
    rule = form_list[1]
    line = form_list[2]
    #int(line) == int(str(real_issue).split(',')[2])
    if not any(
        (rule in str(real_issue) and str(real_issue).split(',')[0].replace("'", "") == benchmark_name)
        for real_issue in real_vulnerabilities
    ):
        FP += 1


# Подсчет ложных положительных
#FP = 1
#len(formatted_issues[formatted_issues['Security Standards'].str.contains('|'.join(real_vulnerabilities_true['cwe'].unique()), na=False)]) - TP

# Расчет метрик
recall = TP / (TP + FN)
precision = TP / (TP + FP)
f1_score = 2*(recall*precision)/(recall+precision)
# Вывод результатов
print(f"Recall: {recall:.2f}")
print(f"precision: {precision:.2f}")
print(f"F1-score: {f1_score:.2f}")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Размер real_vulnerabilities: 200
Recall: 0.70
precision: 0.52
F1-score: 0.59
